In [71]:
import requests
import pandas as pd
from rich import print as printr
import json

In [157]:
# Define query for Fnugg API.
endpoint = 'https://api.fnugg.no/search'
parameters = {
    'size': 150,
    'sourceFields': 'name,contact,resort_open,region,location,weather_zones,conditions,last_updated',
}

In [160]:
# Access Fnug API and confirm successful GET request.
r = requests.get(endpoint, params=parameters)
print(r.status_code)
data_dict = r.json()

200


In [170]:
print ('Number of resorts matching query = ' + str(len(data_dict['hits']['hits'])))
resorts = data_dict['hits']['hits']

Number of resorts matching query = 120


In [162]:
# Save to file for futre use.
with open('fnugg-data.json', 'w') as json_file:
    json.dump(data_dict, json_file, indent=4)

In [163]:
# Reopen from file.
with open('fnugg-data.json', 'r') as json_file:
    data_dict = json.load(json_file)

In [172]:
# Define DataFrame ro capture key details from API reponse.
columns = ['name', 'region', 'resort_open', 'latitude', 'longitude', 'weather_zone_type', 'snow_at_top_current', 'snow_at_bottom_current', 'last_updated']
df = pd.DataFrame(columns=columns)

# Loop through resorts and collect data.
for resort in resorts:
    row = {
        'name': resort['_source']['name'],
        'region': resort['_source']['region'],
        'resort_open': resort['_source']['resort_open'],
        'latitude': resort['_source']['location']['lat'],
        'longitude': resort['_source']['location']['lon'],
        'weather_zone_type': resort['_source']['weather_zones']['type'],
        'snow_at_top_current': resort['_source']['conditions']['current_report']['top']['snow']['depth_slope'],
        'last_updated': resort['_source']['last_updated'],
    }
    if 'bottom' in resort['_source']['conditions']['current_report']: # Some resorts only have 'top' measurements.
        row['snow_at_bottom_current'] = resort['_source']['conditions']['current_report']['bottom']['snow']['depth_slope']
    else:
        row['snow_at_bottom_current'] = None

    df.loc[len(df)] = row


In [173]:
df.shape

(120, 9)

In [174]:
df.to_csv('ski-resorts.csv', index=False)